In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd

In [2]:
DIR = "E:/Kuzushiji/Images/"
train_path = DIR+"train"
validate_path = DIR+"validate"
length = 322
breadth = 200
df = pd.read_csv("train.csv")
df.set_index('image_id', inplace = True)

def img_processor(path):
    img = cv2.imread(path)
    img = cv2.resize(img,(breadth,length), interpolation = cv2.INTER_CUBIC)
    return img

def Batch_generator(DIR):
    while(True):
        for img in os.listdir(DIR):
            im = img_processor(os.path.join(DIR,img))
            ind = img[:-4]
            y_train = df.loc[ind].values
            X_train = np.array(im)
            X_train = X_train.reshape(1,len(X_train),len(X_train[0]),3)
            y_train = y_train.reshape(1,1)
            yield(X_train,y_train)

def Validation_generator(DIR):
    while(True):
        for img in os.listdir(DIR):
            im = img_processor(os.path.join(DIR,img))
            ind = img[:-4]
            y_valid = df.loc[ind].values
            X_valid = np.array(im)
            X_valid = X_valid.reshape(1,len(X_train),len(X_train[0]),3)
            y_valid = y_valid.reshape(1,1)
            yield(X_valid,y_valid)
            

In [4]:
from keras.models import Sequential, Model,load_model
from keras.layers import Add,Input,Dense,Dropout,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda,AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam,Adadelta
from keras.initializers import glorot_uniform

In [5]:
def Conv(model,layers,filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1))) 
        model.add(Conv2D((filters), kernel_size = (3,3), activation = 'relu'))
        
    model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))

    
def FC_block(model,filters):
    model.add(Dense(filters, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
def VGG19():
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=(length,breadth,3)))

    Conv(model,2,64)
    Conv(model,2,128)
    Conv(model,4,256)
    Conv(model,4,512)
    Conv(model,4,512)
    
    model.add(Flatten())
    FC_block(model,4096)
    FC_block(model,4096)
    FC_block(model,1000)
    
    model.add(Dense(1))
    model.add(Activation("softmax"))

    return model

In [6]:
model = VGG19()

model.compile(optimizer = RMSprop(lr=1e-8), loss='mean_squared_error', metrics = ['accuracy'])

W0912 21:22:56.130822   908 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0912 21:22:56.144785   908 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 21:22:56.150805   908 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0912 21:22:56.175702   908 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4267: 

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 322, 200, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 324, 202, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 322, 200, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 324, 202, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 322, 200, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 161, 100, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 163, 102, 64)     

In [8]:
batch_size = 32

steps_per_batch = int(len(os.listdir(train_path))/batch_size)
val_steps_per_batch = int(len(os.listdir(validate_path))/batch_size)

In [9]:
model.fit_generator(
                    Batch_generator(train_path),
                    samples_per_epoch=steps_per_batch,
                    epochs = 5,
                    validation_data=Validation_generator(validate_path),
                    nb_val_samples=val_steps_per_batch,
                    verbose = 1,
                   )

c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=5, validation_data=<generator..., verbose=1, steps_per_epoch=121, validation_steps=49)`
  import sys
W0912 21:23:00.801171   908 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\ten

Epoch 1/5


ValueError: could not convert string to float: 'U+306F 1231 3465 133 53 U+304C 275 1652 84 69 U+3044 1495 1218 143 69 U+3051 220 3331 53 91 U+306B 911 1452 61 92 U+306B 927 3445 71 92 U+306E 904 2879 95 92 U+5DE5 1168 1396 187 95 U+3053 289 3166 69 97 U+4E09 897 3034 121 107 U+306E 547 1912 141 108 U+3084 1489 2675 151 109 U+3068 1561 2979 55 116 U+5DF1 1513 2500 127 117 U+3082 1213 1523 72 119 U+3055 1219 3266 95 124 U+306E 259 2230 68 125 U+306E 1184 2423 169 125 U+4E16 849 2236 163 127 U+7D30 1144 1212 200 128 U+305D 316 3287 57 133 U+4EBA 217 2044 183 135 U+3051 277 2974 112 137 U+308C 201 3423 181 137 U+3060 243 2830 159 143 U+5F37 1479 2034 163 145 U+306E 1497 1567 123 152 U+305F 1164 952 145 153 U+3066 552 1199 97 155 U+4FF3 537 2095 176 155 U+6839 203 1439 184 156 U+304B 1188 2606 156 157 U+8AE7 549 2328 156 159 U+308C 1495 2784 168 159 U+5B50 891 1255 100 164 U+3092 584 2546 117 164 U+53CA 849 1588 151 164 U+8005 1192 2198 133 169 U+305A 889 1763 103 171 U+907F 513 945 181 171